<a href="https://colab.research.google.com/github/RahulManavalan/Machine-Learning/blob/master/Linear_regression_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
pip install tensorflow==1.0

In [0]:
import pandas as pd 
import numpy as np 

In [0]:
def read_goog_sp500_data():
  googFile = "/GOOG.csv"
  spFile = "/content/^IXIC.csv"

  goog = pd.read_csv(googFile,sep=",",usecols=[0,5],names=["Date","Goog"],header=0)
  sp = pd.read_csv(spFile,sep=",",usecols=[0,5],names=["Date","SP500"],header=0)


  goog["SP500"] = sp["SP500"]

  goog["Date"] = pd.to_datetime(goog["Date"],format="%Y-%m-%d")
  goog = goog.sort_values(["Date"],ascending=[True])

  returns = goog[[key for key in dict(goog.dtypes) if dict(goog.dtypes)[key] in ["float64","int64"]]].pct_change() 

  xData = np.array(returns["SP500"])[1:]
  yData = np.array(returns["Goog"])[1:]

  return(xData,yData) 

In [0]:
xData , yData = read_goog_sp500_data()

In [0]:
import tensorflow as tf 

In [0]:
W = tf.Variable(tf.zeros([1,1]))
b = tf.Variable(tf.zeros([1]))

x = tf.placeholder(tf.float32,[None,1])
Wx = tf.matmul(x,W) 
y = Wx + b 

W_hist = tf.summary.histogram("weights",W)
b_hist = tf.summary.histogram("biases",b) 
y_hist = tf.summary.histogram("y" , y) 
y_ = tf.placeholder(tf.float32 , [None,1]) 



In [0]:
cost = tf.reduce_mean(tf.square(y - y_))
cost_hist = tf.summary.histogram("cost",cost)
train_step_adagrad = tf.train.FtrlOptimizer(1).minimize(cost) 

In [0]:
def trainwithoneEpoch(steps,train_step): 
  init = tf.global_variables_initializer() 

  with tf.Session() as sess:
    sess.run(init) 
    merged_summary = tf.summary.merge_all() 
    writer = tf.summary.FileWriter("./linear_regression_demo1",sess.graph)

    for i in range(steps): 
      xs = np.array([[xData[i%len(xData)]]])
      ys = np.array([[yData[i%len(yData)]]])

      feed = {x:xs , y_:ys}

      sess.run(train_step,feed_dict=feed)
      result = sess.run(merged_summary,feed_dict=feed) 
      writer.add_summary(result,i) 

 
      if (i+1)%1000 == 0 : 
        print("After %d iterations :" %i)

        print("W %f" % sess.run(W) )
        print("b %f" % sess.run(b) )

        print("Cost function %f" % sess.run(cost,feed_dict=feed))

        writer.close()

In [0]:
data_size = len(xData) 
def trainwithMultiplePointsperEpoch(steps,train_step,batch_size):
  init = tf.global_variables_initializer() 
  with tf.Session() as sess: 
    sess.run(init) 
    merged_summary = tf.summary.merge_all() 
    writer = tf.summary.FileWriter("./linear_regression_demo2",sess.graph)

    for i in range(steps):
      if data_size == batch_size:
        batch_size_idx = 0 
      elif data_size < batch_size:
        raise ValueError("dataset_size must be greater than the batch size")
      else : 
        batch_size_idx = (i*batch_size)%(data_size)
      batch_end_idx = batch_size_idx + batch_size 

      batch_xs = xData[batch_size_idx:batch_end_idx] 
      batch_ys = yData[batch_size_idx:batch_end_idx]

      feed = {x:batch_xs.reshape(-1,1) , y_:batch_ys.reshape(-1,1)}
      sess.run(train_step , feed_dict=feed)

      #result = sess.run(merged_summary,feed_dict=feed) 
      #writer.add_summary(result,i) 

 
      if (i+1)%1000 == 0 : 
        print("After %d iterations :" %i)

        print("W %f" % sess.run(W) )
        print("b %f" % sess.run(b) )

        print("Cost function %f" % sess.run(cost,feed_dict=feed))

        writer.close()

      

In [87]:
trainwithMultiplePointsperEpoch(5000,train_step_adagrad,len(xData))

After 999 iterations :
W 1.013051
b 0.004040
Cost function 0.003492
After 1999 iterations :
W 1.013051
b 0.004040
Cost function 0.003492
After 2999 iterations :
W 1.013051
b 0.004040
Cost function 0.003492
After 3999 iterations :
W 1.013051
b 0.004040
Cost function 0.003492
After 4999 iterations :
W 1.013051
b 0.004040
Cost function 0.003492
